# Super Lucky Palindromes Notebook

Original problem : https://www.spoj.com/problems/CTPLUCKY/

This notebook will present my work on the problem I had really fun solving for the past two months.
At first, I solved this problem with a naive algorithm (v1 and v2) but they were not fast enough to solve the 10^18th index which the problem's statement wants.

I did a break of 2 months to came back with the idea of solving it. <br>
Which I did.

In this notebook you'll find the predata code which helps to get initial parameters for v1, v2 and v3. <br>
All algorithm version I did to solve the problem. <br>
At the end a time comparison between these algorithm which will suprise you I think :)

Little vocabulary :
- A lucky number is a number that is only made of "4" and "7"
- A super lucky number is a lucky number where either its amount of "4" or "7" is itself a lucky number
- A super lucky palindrome is a super lucky number which is also a palindrome
- Super Lucky Palindrome will be called SLP

## Predata

This predata code computes values to make a slightly faster algorithm.<br>
These values are always the same, that's why I can store them in array.

The statement says I only need to get index until 10^18th one.<br>
With combination formula I know how much possibilities exist with a given length number.

This code return 3 differents values :
- Possible length which are (4, 7, 44, 47, 74, 77, 444)
- Amount of possibilities for each length
- Pair numbers. To know all possible amount of "4" and "7" one side of a palindrome can have in order to make a SLP (I call it Pair numbers because if you know the amount of "4" you can get the amount of "7", they work by pair)

It works because I "simulate" the complete palindrome number from only one side. So I need to check if the amount of "4" or either the amount of "7" will be a lucky number.

In [ ]:
import scipy.special

def is_lucky(length):
    if any(x in str(length) for x not in "47"):
        return 0
    return 1

def get_pair(length):
    pair = []
    limit = length // 2
    is_odd = length % 2
    
    for amount_4 in range(0, limit+1):
        if is_lucky(amount_4*2+is_odd) == 1 or is_lucky(amount_4*2) == 1 or \
        is_lucky(length-amount_4*2-is_odd) == 1 or is_lucky(length-amount_4*2) == 1:
            pair.append(amount_4)
    return pair

def get_possibilities(length, pair):
    total = 0
    for i in pair:
        total += scipy.special.comb(length, i, exact=True)
    return total

In [ ]:
for i in range(4,445):
    if is_lucky(i) == 1:
        pair = get_pair(i)
        print(i, pair, int(get_possibilities(i//2, pair)))

## v1 : Naive algorithm

The idea is simple, create every possibilities and check for every possibility if its a SLP.<br>
The numbers I'm working with are palindromes which implies that I can work with only one side. Once one side is calculated, I just have to reverse it and adding it to itself and it will makes a palindrome number.<br>

One little issue with this are odd numbers. Odd numbers have one more digit I need to find to create a SLP<br>
To do it I simulate this "if I create a palindrome number with the side I calculated and I add a "4" at the middle, is it a super lucky number ? Same with adding a "7" instead"

I also discovered that once you've found half of the solutions for a given length, you just have to change the "4" by "7" and "7" by "4" to create the opposite solution. You can look at query 6 and 7, 5 and 8, 4 and 9 ... you'll see they are opposite<br>
Compute the ten first possibilities and you'll see.<br>
This way I can optimize the algorithm even more with the help of "self.reverse" which will exchange the "4" by "7".

About the recursive function :<br>
Instead of having an empty array and placing either a "4" or a "7" inside it I create an all "4" (or "7") array and I just change the values if I have to. This way I can create a more generic code.

In [ ]:
import copy

class lucky():
    def __init__(self): 
        self.length_list = [4, 7, 44, 47, 74, 77, 444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0
        self.limit = 0
        self.query = 0
        self.cursor = 1
        self.reverse = False
        self.solution = ""
    
    def get_solution(self, query):
        if query <= 0:
            print("lol k")
            return None
        if query > sum(self.range_list):
            print("Number too big")
            return None
        
        self.index = 0
        while query - self.range_list[self.index] > 0:
            query -= self.range_list[self.index]
            self.index += 1
        
        self.reverse = False
        if query - (self.range_list[self.index]//2) > 0:
            query = self.range_list[self.index] - query + 1
            self.reverse = True
        
        self.limit = self.length_list[self.index]//2
        self.query = query
        self.cursor = 0
        self.solution = ""
        if self.reverse == False:
            self.generate(["4"]*self.limit) #Everything
        else:
            self.generate(["7"]*self.limit) #Everything
        
        return "".join(self.solution)
        
    def put(self, res, pos):
        if res[pos] == "4":
            res[pos] = "7"
        else:
            res[pos] = "4"
        return res
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query:
            return
        if pos == self.limit:
            if num4 in self.pair_list[self.index] or num7 in self.pair_list[self.index]:
                self.cursor += 1
                if self.cursor == self.query:
                    if self.length_list[self.index]%2 == 0:
                        self.solution = res + res[::-1]
                    else:
                        if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                            self.solution = res + ['4'] + res[::-1]
                        else:
                            self.solution = res + ['7'] + res[::-1]
                        if self.reverse == True:
                            self.put(self.solution, self.limit)
        else:
            self.generate(res, pos + 1, num4 + 1, num7)
            self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1)

In [ ]:
v1 = lucky()
for i in range(1,20):
    print(v1.get_solution(i), i)

## v2 : Naive algorithm - Optimized

The only thing that change with the previous version is the recursive part.

The previous algorithm had a big flaw because I'm computing every exiting possibilities and then checking if the current result can be a SLP.<br>
I created the function "is_possible" that can fix the issue by detecting as soon as possible if the current solution i'm computing will give a SLP or not.

The idea is simple : With the current state of the result, does a solution still exists if I add a "4" (or a "7") ?<br>
Refering to the pair numbers, I know how much "4" or "7" I need to have at the end.<br>
So if I don't have enough space left to write a "4" and create a SLP, it is useless to add a "4".

Thanks to this I'm sure that, once every digit is written, I do have an existing solution.<br>
I still have to compute the middle digit of odd numbers length tho.

In [ ]:
import copy

class lucky_v2():
    def __init__(self): 
        self.length_list = [4,7,44,47,74,77,444]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246514075237194788708873582383079842]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38]]
        
        self.index = 0
        self.limit = 0
        self.query = 0
        self.cursor = 1
        self.reverse = False
        self.solution = ""
    
    def get_solution(self, query):
        if query <= 0:
            print("lol k")
            return None
        if query > sum(self.range_list):
            print("Number too big")
            return None
        
        self.index = 0
        while query - self.range_list[self.index] > 0:
            query -= self.range_list[self.index]
            self.index += 1
        
        self.reverse = False
        if query - (self.range_list[self.index]//2) > 0:
            query = self.range_list[self.index] - query + 1
            self.reverse = True
        
        self.limit = self.length_list[self.index]//2
        self.query = query
        self.cursor = 0
        self.solution = ""
        if self.reverse == False:
            self.generate(["4"]*self.limit) #Everything
        else:
            self.generate(["7"]*self.limit) #Everything
        
        return "".join(self.solution)
        
    def put(self, res, pos):
        if res[pos] == "4":
            res[pos] = "7"
        else:
            res[pos] = "4"
        return res
    
    def is_possible(self, amount, space, values):
        idx = 0
        while amount >= values[idx]:
            idx += 1
        if amount + space >= values[idx]:
            return 1
        return 0
    
    def generate(self, res, pos=0, num4=0, num7=0):
        if self.cursor > self.query:
            return
        if pos == self.limit:
            self.cursor += 1
            if self.cursor == self.query:
                if self.length_list[self.index]%2 == 0:
                    self.solution = res + res[::-1]
                else:
                    if num4*2 + 1 in self.length_list or num7*2 in self.length_list:
                        self.solution = res + ['4'] + res[::-1]
                    else:
                        self.solution = res + ['7'] + res[::-1]
                    if self.reverse == True:
                        self.put(self.solution, self.limit)
        else:
            if self.is_possible(num4, self.limit-pos, self.pair_list[self.index]) == 1:
                self.generate(res, pos + 1, num4 + 1, num7)
            if self.is_possible(num7, self.limit-pos, self.pair_list[self.index]) == 1:
                self.generate(self.put(copy.copy(res), pos), pos + 1, num4, num7 + 1)

In [ ]:
v1 = lucky()
v2 = lucky_v2()
for i in range(1,20):
    print(i)
    if v1.get_solution(i) != v2.get_solution(i):
        print(v1.get_solution(i))
        print(v2.get_solution(i))

## v3 : Mathematical approach - Limited

I will not explain in detail every part of the code here (check the github) but just the idea in general.

The idea of this algorithm is like this :<br>
I know how much possibilities there is for a given length (refer you to predata code) I can know how much length I need to have before putting the first "7".<br>
Exemple : Let's have this parameters, length = 4, pair = [0,2], query = 8. The possibilities are :<br>
- 0 > 4444
- 1 > 4477
- 2 > 4747
- 3 > 4774
- 4 > 7447
- 5 > 7474
- 6 > 7744

They are seven of them. The amount of these possibilities is calculated with binomial function (check predata for this).<br>
My query is currently 8, I am now sure that the first "7" to put will not be at one of these indexes. But, If I calculate with a length of 5 it gives this :<br>
- 0 > 44444
- 1 > 44477
- 2 > 44747
- 3 > 44774
- 4 > 47447
- 5 > 47474
- 6 > 47744
- 7 > 74447
- 8 > 74474
- 9 > 74744
- 10> 77444

The query 8 <= 10 so I am sure the first "7" to put will be contained in one of the last 4 solutions.<br>
So now I do a little trick, I decrease the value of query by the amount of possibilities from length-1. In others terms, I "remove" the possibilities which doesn't have a "7" at their last position (or first position depending of your vocabulary).<br>
Mind that we also need to decrease the pair values because we just put one "7". That means the pair[0] won't be possible anymore !<br>
And so we can continue with the same logic. Find how much possibilities exists for all length to know exactly when the next "7" will be placed at.

One last thing, this algorithm doesn't work with very large numbers. It had an issue I had trouble with.<br>
The algorithm was working perfectly for first indexes (until ~10^18), why suddenly it would stops working for big length number ???<br>
I found out why, it was the use of float numbers which are not precise enough for big length SLP.

That's why the v4 exists :)

Little note : the solution is also fast enough to skip the "reverse" feature

In [ ]:
import copy
import scipy.special

class lucky_v3():
    def __init__(self):
        self.length_list = [4, 7, 44, 47, 74, 77, 444, 447, 474, 477, 744, 747, 774, 777]
        self.range_list = [2, 8, 464, 4096, 18728400854, 75422540336, 3949534246513022733692774166275217636196352, 27912724205136969367829202079257831965458432, 54901863838981370331031720163633347084419072, 408867219021824519558760381286789756865216512, 731060835093268059719935428380319894767978536241103340770302477220618973863638154274876419272070389261402112, 3028768728981208771138392332161427475712344495983372066026850069568299204087095837473068158977043396135223296, 383767495066681352637998366663200070705183074877734592607500475472627707939626649121930829298776144224727016144896, 1564887013153773591096691217957484965016329465244742414975904717557119142894155366820818937587853276546953480503296]
        self.pair_list = [[0,2], [0,1,2,3], [0,2,20,22], [0,1,2,3,20,21,22,23], [0,2,15,22,35,37], [0,1,2,3,15,16,22,23,35,36,37,38], [0,2,22,37,185,200,220,222], [0,1,2,3,22,23,37,38,185,186,200,201,220,221,222,223], [0,2,15,22,37,200,215,222,235,237], [0,1,2,3,15,16,22,23,37,38,200,201,215,216,222,223,235,236,237,238], [0,2,22,37,135,150,222,237,335,350,370,372], [0,1,2,3,22,23,37,38,135,136,150,151,222,223,237,238,335,336,350,351,370,371,372,373], [0,2,15,22,37,150,165,222,237,350,365,372,385,387], [0,1,2,3,15,16,22,23,37,38,150,151,165,166,222,223,237,238,350,351,365,366,372,373,385,386,387,388]]
        
        self.limit = 0
        self.seven_amount = 0
        self.pair = []
        self.solution = ""
    
    def get_solution(self, query):
        index = 0
        while query - self.range_list[index] > 0:
            query -= self.range_list[index]
            index += 1
        
        query -= 1
        self.limit = self.length_list[index]//2
        self.seven_amount = 0
        self.pair = copy.copy(self.pair_list[index])
        self.solution = ["4"] * self.limit
        
        self.slp_algorithm(query) #everything
        
        if self.length_list[index]%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else:
            if self.seven_amount*2+1 in self.length_list or (self.limit-self.seven_amount)*2 in self.length_list:
                self.solution = self.solution[::-1] + ["7"] + self.solution
            else:
                self.solution = self.solution[::-1] + ["4"] + self.solution
        
        return "".join(self.solution)

    def get_comb(self, length, offset=0):
        comb = []
        for pos in range(1, length+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += scipy.special.comb(pos, i)
            comb.append(total)
        return comb
    
    def slp_algorithm(self, cursor, pos=0, depth=0):
        if self.pair[0] - depth < 0:
            del self.pair[0]
        if cursor < 0:
            return
        if cursor == 0:
            for i in range(self.pair[0] - depth):
                self.solution[i] = "7"
                self.seven_amount += 1
            return
        
        comb = self.get_comb(self.limit - pos, depth)
        idx = 0
        while idx < self.limit - depth and cursor >= comb[idx]:
            idx += 1
        self.solution[idx] = "7"
        self.seven_amount += 1
        cursor -= comb[idx-1]
        self.slp_algorithm(cursor, self.limit - idx, depth + 1)

In [ ]:
yes = lucky_v3()
for i in range(1, 25):
    print(yes.get_solution(i), i)

## v4 Mathematical approach - No limit (Non recursive)

It is poorly coded (imo) but the algorithm do the job very well. <br>
I have not really anything to say.

This v4 is basically v3 using only python integers and with a non recursive function to solve the problem.<br>
That means you can compute query 10^100 it will works

What ?<br>
Yeah, 10^100<br>
You wanna try ? How much time do you think it takes ?<br>
Imo, I think it is not enough. I need to optimize until 10^10000

I have coded a verbose feature if you're curious about the inside work of the algorithm for very very large number

IMPORTANT NOTE : While trying to solve 10^1000 number I figured out that the algorithm could be faster in two ways.<br>
First, I do useless computation to create the comb array. Imagine the first seven to put being at middle index, there is a whole half of the comb array that won't be used. I have to first try a position AND THEN calculate its amount of possibilities.<br>
Second, I need to use binary search algorithms. I have a sorted array (comb), so I need to compute the index at half of length and then, if my cursor is above I need to compute the 3/4 index, if not it will be the 1/4. And so on and so on.

In [ ]:
import scipy.special
import time
from sys import stdout

class lucky_v4():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.time_elapsed = 0
        
        self.query = 0
        self.cursor = 0
        self.length = 0
        self.limit = 0
        self.pair = []
        self.seven_amount = 0
        self.total = 0
        self.solution = ""
        
    def verbose_print(self, text_index): #ignore this, it's just display
        if self.verbose == False:
            return
        
        if text_index == 0:
            self.time_elapsed = time.time()
            print("Algorithm started")
        if text_index == 1:
            stdout.write("\rSolution will have a length of %d. Current length of amount of possibilities is %d" % (self.length, len(str(self.total))))
        if text_index == 2:
            print("\n")
            print("Now computing solution for query", self.query)
            print()
        if text_index == 3:
            stdout.write("\rCursor currently at position %d/%d (%d%%). Amount of seven put %d. Time elapsed %f" \
                         % (self.limit-self.space, self.limit, int((self.limit-self.space)/self.limit*100), self.seven_amount, time.time() - self.time_elapsed))
        if text_index == 4:
            print("\n")
            print("Algorithm finished")
            print("Solution have a length of", len(self.solution), "and have", self.solution.count("4"), "for and", self.solution.count("7"), "seven")
            print()
            print("Total time elapsed", time.time() - self.time_elapsed)
        
    def is_lucky(self, length):
        if any(x in str(length) for x in "01235689"):
            return 0
        return 1
    
    def get_palindrome_pair(self, length):
        pair = []
        is_odd = length % 2
        limit = length // 2
        total = 0
        for x in range(0, limit+1):
            if self.is_lucky(x*2+is_odd) == 1 or self.is_lucky(x*2) == 1 or \
            self.is_lucky(length-x*2-is_odd) == 1 or self.is_lucky(length-x*2) == 1:
                pair.append(x)
        return pair
    
    def get_lucky_from_index(self, index):
        binary = "{0:b}".format(index+2)
        res = 0
        for idx, i in enumerate(binary[1:]):
            res += (4 if i == "0" else 7) * 10**(len(binary)-idx-2)
        return res
        
    def get_solution(self, query):
        self.verbose_print(0)
        
        self.query = query
        index = 0
        while self.query > 0:
            self.total = 0
            self.length = self.get_lucky_from_index(index)
            self.pair = self.get_palindrome_pair(self.length)
            self.limit = self.length // 2
            for i in self.pair:
                self.total += scipy.special.comb(self.limit, i, exact=True)
            self.query -= self.total
            index += 1
            self.verbose_print(1)
        
        self.query += self.total - 1
        self.seven_amount = 0
        self.verbose_print(2)
        self.solution = self.slp_algorithm() #everything
        
        if self.length%2 == 0:
            self.solution = self.solution[::-1] + self.solution
        else:
            if self.is_lucky(self.seven_amount*2+1) == 1 or self.is_lucky((self.limit-self.seven_amount)*2) == 1:
                self.solution = self.solution[::-1] + "7" + self.solution
            else:
                self.solution = self.solution[::-1] + "4" + self.solution
                
        self.verbose_print(4)
        return self.solution
    
    def get_comb(self, offset=0):
        comb = []
        for pos in range(1, self.limit+1):
            total = 0
            for i in self.pair:
                i -= offset
                if i <= pos:
                    total += scipy.special.comb(pos, i, exact=True)
            comb.append(total)
        return comb
    
    def slp_algorithm(self):
        self.cursor = self.query
        self.space = self.limit
        self.solution = ["4"] * self.space
        amount = 0

        while self.cursor > 0:
            comb = self.get_comb(amount)
            idx = 0
            while idx < self.limit - self.seven_amount and self.cursor >= comb[idx]:
                idx += 1
            self.solution[idx] = "7"
            self.seven_amount += 1
            
            if idx <= 0:
                break
            self.cursor -= comb[idx-1]
            self.space = idx
            amount += 1
            self.verbose_print(3)
        
        if self.cursor == 0:
            idx = 0
            while self.pair[idx] < amount:
                idx += 1
            for i in range(self.pair[idx] - amount):
                self.solution[i] = "7"
                self.seven_amount += 1
        self.space = 0
        self.verbose_print(3)

        return "".join(self.solution)

In [ ]:
query = 10**200
lucky_v4(True).get_solution(query)

## Time comparison

Here you'll find a little script that will execute every algorithm and will display the amount of time each algorithm take to find the solution for different index.<br>
Indexes are selected for a good comparison between each algorithm.

Mind that v3 is wrong for big numbers tho because its working with float. outside of that, v3 and v4 are almost the exact same.

In [ ]:
import time

v1 = lucky()
v2 = lucky_v2()
v3 = lucky_v3()
v4 = lucky_v4()
for i in [20, 50, 100, 250, 2500, 10**5, 10**6]:
    for v, algo in enumerate([v1, v2, v3, v4]):
        start = time.time()
        #algo.get_solution(i)
        print("query", i, "with", "v"+str(v+1), "in", str(int((time.time()-start)*1000))+"ms")
    print()

print("------------------\n")

for i in range(1, 11):
    start = time.time()
    v4.get_solution(10**(10*i))
    print("query", "10^"+str(i), "with v4 in", str(int((time.time()-start)*1000))+"ms")